In [1]:
#import libraries


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 1.1 Load and Inspect Data 

stats_df = pd.read_csv('../data/stats.csv')
results_df = pd.read_csv('../data/results.csv')

print("Stats shape:", stats_df.shape)
print("Results shape:", results_df.shape)

# Display first rows and info
stats_df.head()
stats_df.info()
results_df.head()
results_df.info()



Stats shape: (240, 42)
Results shape: (4560, 6)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 42 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   team                  240 non-null    object 
 1   wins                  240 non-null    float64
 2   losses                240 non-null    float64
 3   goals                 240 non-null    float64
 4   total_yel_card        240 non-null    float64
 5   total_red_card        240 non-null    float64
 6   total_scoring_att     240 non-null    float64
 7   ontarget_scoring_att  240 non-null    float64
 8   hit_woodwork          240 non-null    float64
 9   att_hd_goal           240 non-null    float64
 10  att_pen_goal          240 non-null    float64
 11  att_freekick_goal     240 non-null    float64
 12  att_ibox_goal         240 non-null    float64
 13  att_obox_goal         240 non-null    float64
 14  goal_fastbreak        240 

In [3]:
# 1.2 Handle Missing Values


# Check missingness
print("Stats missing values:\n", stats_df.isnull().sum())
print("\nResults missing values:\n", results_df.isnull().sum())

# Stats: some columns have empty cells (e.g., outfielder_block in early seasons)
# We'll impute numeric columns with median (robust to outliers)
numeric_cols_stats = stats_df.select_dtypes(include=[np.number]).columns
imputer = SimpleImputer(strategy='median')
stats_df[numeric_cols_stats] = imputer.fit_transform(stats_df[numeric_cols_stats])

# For categorical columns like 'team', 'season' – no missing values observed.
# If any appear, we could drop rows or fill with mode, but not needed here.

# Results: no missing values except possibly home_goals/away_goals? Check.
# All goals appear present; we'll keep as is.


Stats missing values:
 team                     0
wins                     0
losses                   0
goals                    0
total_yel_card           0
total_red_card           0
total_scoring_att        0
ontarget_scoring_att     0
hit_woodwork             0
att_hd_goal              0
att_pen_goal             0
att_freekick_goal        0
att_ibox_goal            0
att_obox_goal            0
goal_fastbreak           0
total_offside            0
clean_sheet              0
goals_conceded           0
saves                   20
outfielder_block         0
interception             0
total_tackle             0
last_man_tackle          0
total_clearance          0
head_clearance          20
own_goals                0
penalty_conceded         0
pen_goals_conceded       0
total_pass               0
total_through_ball      20
total_long_balls         0
backward_pass           80
total_cross              0
corner_taken             0
touches                  0
big_chance_missed       80
clear

In [4]:
# 1.3 Detect and Treat Outliers

# Function to cap outliers at given percentiles
def cap_outliers(df, columns, lower_percentile=0.01, upper_percentile=0.99):
    for col in columns:
        lower = df[col].quantile(lower_percentile)
        upper = df[col].quantile(upper_percentile)
        df[col] = df[col].clip(lower, upper)
    return df

# Select numeric stats columns that might have extreme values
cols_to_cap = ['goals', 'total_scoring_att', 'total_pass', 'touches']
stats_df = cap_outliers(stats_df, cols_to_cap)



In [5]:
# 1.4 Correct Data Types


# Ensure season is string, then extract start year as integer
stats_df['season'] = stats_df['season'].astype(str)
stats_df['season_start'] = stats_df['season'].str[:4].astype(int)

results_df['season'] = results_df['season'].astype(str)
results_df['season_start'] = results_df['season'].str[:4].astype(int)

# Convert team names to string
stats_df['team'] = stats_df['team'].astype(str)
results_df['home_team'] = results_df['home_team'].astype(str)
results_df['away_team'] = results_df['away_team'].astype(str)


In [8]:
# 1.5 Logical Consistency Checks 


# Verify wins+losses+draws = 38 for each team-season
# 1. Calculate the 'draws' column first since it is missing from the CSV
stats_df['draws'] = 38 - (stats_df['wins'] + stats_df['losses'])

# 2. Now run your consistency check
stats_df['total_matches'] = stats_df['wins'] + stats_df['losses'] + stats_df['draws']

inconsistent = stats_df[stats_df['total_matches'] != 38]
if len(inconsistent) > 0:
    print("Teams with match count != 38:")
    print(inconsistent[['team', 'season', 'wins', 'losses', 'draws']])
else:
    print("All teams have a consistent match count of 38.")

# 3. Check that goals <= total_scoring_att
assert (stats_df['goals'] <= stats_df['total_scoring_att']).all()

All teams have a consistent match count of 38.


In [9]:
# 1.6 Feature Sanity and Creation 


# Create points column
stats_df['points'] = 3*stats_df['wins'] + stats_df['draws']

# Create goal difference
stats_df['goal_diff'] = stats_df['goals'] - stats_df['goals_conceded']

# Create ratios (avoid division by zero)
stats_df['shot_accuracy'] = stats_df['ontarget_scoring_att'] / stats_df['total_scoring_att'].replace(0, np.nan)
stats_df['pass_completion'] = stats_df['total_pass'] / stats_df['touches'].replace(0, np.nan)  # rough proxy